#### Codigo que carga data de excel a la tabla de sql

In [1]:
import os
import pandas as pd
import zipfile
import pyodbc
import requests
import io
from datetime import datetime, timedelta
import sqlalchemy
import urllib

In [2]:
server = 'DARCCVWSQL19'
#server = 'DESKTOP-37ESKFT\SQLEXPRESS'
database = 'TAPI'
tabla = 'DiarioTest'

In [3]:
# Test conexión
connection_string = f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes;'
 
try:
    # Intentar establecer la conexión
    connection = pyodbc.connect(connection_string)
 
    # Si no hay errores, la conexión se ha establecido correctamente
    print("Conexión exitosa a la base de datos SQL Server.")
 
    # Cerrar la conexión
    connection.close()
except pyodbc.Error as e:
    # Manejar los errores de conexión
    print(f"Error al conectar a la base de datos: {e}")

Conexión exitosa a la base de datos SQL Server.


In [4]:
#Escribir el path completo al excel o en la raiz del notebook

#excel_path = r"C:\Users\jony\OneDrive - Pampa Energia\CDD - Tablero de Análisis de Perdidas de Ingresos\Documentación\2-Fuente de Datos\df_POSOP_diario.xlsx"
excel_path = r"C:\Users\jadurian\OneDrive - Pampa Energia\CDD - Tablero de Análisis de Perdidas de Ingresos\Codigo\Consulta diaria\df_POSOP_diario.xlsx"
name_excel = os.path.basename(excel_path)
print(name_excel)

# Leer los datos de Excel
df = pd.read_excel(excel_path, sheet_name="Sheet1")

df_POSOP_diario.xlsx


In [8]:
df.columns

Index(['FECHA', 'GRUPO', 'HORA', 'ARL', 'ESTADO_OPE', 'PI_SOTR', 'ARL_ECON',
       'EST_REMUN', 'ENERGIA', 'PRECIO_NODO', 'PR_REM_ENERGIA', 'SMEC',
       'POT_DISP', 'POT_OPE_CPO', 'COMPRA_SPOT', 'SCTD', 'SCO', 'PIND',
       'PINDPROG', 'PINDFORZ', 'POT_DISP_GAS', 'POT_DISP_RESERVA',
       'GAS_NOMINADO', 'COSTO_406', 'REM_ADICIONAL', 'DESP_ECON', 'CCM', 'CFO',
       'CGN', 'CGO', 'POT_DISP_R19', 'FACTOR_KM', 'HORA_REPRESENTATIVA',
       'ENERG_GEN_TRANSAC', 'ENERG_OPE_TRANSAC', 'SOBREC_FORZ_PROPIO',
       'PGENE_RES354', 'CGU_PROPIO', 'CGU_MEM', 'CGU_GAS_CEDIDO'],
      dtype='object')

In [7]:
# En caso de tener que renombrar o re ubicar columnas:

# Renombrar la columna
#df.rename(columns={'Fecha_dia': 'FECHA'}, inplace=True)

# Posicionar la nueva columna como la primera columna
nuevas_columnas = ['FECHA'] + [col for col in df if col != 'FECHA']
df = df.reindex(columns=nuevas_columnas)

In [9]:
#Cargamos el Dataframe a la tabla de sql con sqlalchemy

tabla = 'DiarioTest'

quoted = urllib.parse.quote_plus(connection_string)

engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

df.to_sql(tabla, schema='dbo', con=engine, if_exists='replace')

14